In [33]:
import pandas as pd
import os
from tqdm import tqdm
from utils import avg, evidence_to_mask

def to_data_df(df, data_dir):
    data_df = []
    columns = ['text', 'classification', 'rationale', 'query']
    for i in tqdm(range(len(df))):
        df_row = df.loc[i]
        text_id = df_row['docids'][0]
        query = df_row['query']
        evidence_list = df_row['evidences']
        if evidence_list:
            evidence_list = [x for xx in evidence_list for x in xx]
        classification = df_row['classification']
        
        file = f'{data_dir}/docs/{text_id}'
        if file[-1] == '.': file = file[:-1] + '_'
#         print(file)
        if os.path.isfile(file):
            f = open(file, 'r', encoding="utf-8") 
            text = ''
            for line in f.readlines():
                text += line.rstrip() + ' '
        else:
            print("???")      
        
        tokens = text.split()
        rationale_mask = evidence_to_mask(tokens, evidence_list)
        
        # joining text and query with [SEP]
        QA = f"{query} [SEP] {text}"
        rationale_mask = [1]*(len(query.split())+1) + rationale_mask
        
        data_df.append([QA, classification, rationale_mask, query])
    data_df = pd.DataFrame(data_df, columns=columns)
    return data_df

#     data_df_shuffled=data_df.sample(frac=1).reset_index(drop=True)
#     return data_df_shuffled

In [34]:
dataset = "fever"

In [35]:
data_dir = f'../data/{dataset}'
out_dir = f'others/{dataset}'
train = pd.read_json(f'{data_dir}/train.jsonl', lines=True)
test = pd.read_json(f'{data_dir}/test.jsonl', lines=True)
val = pd.read_json(f'{data_dir}/val.jsonl', lines=True)

In [36]:
train_data_df = to_data_df(train, data_dir)
train_data_df.to_csv(f"{out_dir}/train.csv",index_label="id")
test_data_df = to_data_df(test, data_dir)
test_data_df.to_csv(f"{out_dir}/test.csv",index_label="id")
val_data_df = to_data_df(val, data_dir)
val_data_df.to_csv(f"{out_dir}/val.csv",index_label="id")

100%|█████████████████████████████████████████████████████████████████████████████| 6122/6122 [00:06<00:00, 905.69it/s]


# SANITY CHECK

In [12]:
def rationale_check(text,rationale):
    tokens = text.split()
    out = ""
    for i, b in enumerate(rationale):
        if b:
           out += tokens[i] + " "
    return out

In [10]:
data_df = test_data_df
import json

def reduce_by_alpha(text, rationale, fidelity_type="sufficiency"):
    reduced_text = ""
    # whitespace tokenization
    tokens = text.split()

    for idx in range(len(tokens)):
        try:
            if fidelity_type == "sufficiency" and rationale[idx] >= 0.5:
                reduced_text = reduced_text + tokens[idx] + " "
            elif fidelity_type == "comprehensiveness" and rationale[idx] < 0.5:
                reduced_text = reduced_text + tokens[idx] + " "
        except Exception as e:
            if fidelity_type == "comprehensiveness":
                reduced_text = reduced_text + tokens[idx] + " "

    # removed the last space from the text
    if len(reduced_text) > 0:
        reduced_text = reduced_text[:-1]

    return reduced_text

data_df = data_df[data_df['rationale'].notna()]
data_df.reset_index(drop=True, inplace=True)


data_df["sufficiency_text"] = data_df[
    ["text", "rationale"]].apply(lambda s: reduce_by_alpha(*s, fidelity_type="sufficiency"), axis=1)
data_df["comprehensiveness_text"] = data_df[
    ["text", "rationale"]].apply(lambda s: reduce_by_alpha(*s, fidelity_type="comprehensiveness"), axis=1)

In [38]:
data_df['sufficiency_text'][0]

'1,000 genomes project enables mapping of genetic sequence variation consisting of rare variants with larger penetrance effects than common variants. [SEP] We propose as an alternative explanation that variants much less common than the associated one may create " synthetic associations " by occurring , stochastically , more often in association with one of the alleles at the common site versus the other allele . We show that they are not only possible , but inevitable , and that under simple but reasonable genetic models , they are likely to account for or contribute to many of the recently identified signals reported in genome-wide association studies . In conclusion , uncommon or rare genetic variants can easily create synthetic associations that are credited to common variants , and this possibility requires careful consideration in the interpretation and follow up of GWAS signals .'

In [13]:
rationale_check(train_data_df.iloc[2]['text'],train_data_df.iloc[2]['rationale'])

'IZombie premiered in 2015. [SEP] The series premiered on March 17 , 2015 . '

In [40]:
train

,annotation_id,classification,docids,evidences,query,query_type
0,10,REFUTES,[Ireland],"[[{'docid': 'Ireland', 'end_sentence': 7, 'end...",Ireland does not have relatively low-lying mou...,NaN
1,100002,SUPPORTS,[Taylor_Schilling],"[[{'docid': 'Taylor_Schilling', 'end_sentence'...",The drama Dark Matter stars Taylor Schilling.,NaN
2,100004,SUPPORTS,[IZombie_-LRB-TV_series-RRB-],"[[{'docid': 'IZombie_-LRB-TV_series-RRB-', 'en...",IZombie premiered in 2015.,NaN
3,100005,SUPPORTS,[Ronald_Reagan],"[[{'docid': 'Ronald_Reagan', 'end_sentence': 1...",Ronald Reagan had a nationality.,NaN
4,100006,SUPPORTS,[Samoa_Joe],"[[{'docid': 'Samoa_Joe', 'end_sentence': 1, 'e...",Samoa Joe wrestles professionally.,NaN
...,...,...,...,...,...,...
97952,99990,SUPPORTS,[The_Simpsons],"[[{'docid': 'The_Simpsons', 'end_sentence': 12...",The Simpsons have aired on Fox for 29 seasons.,NaN
97953,99992,REFUTES,[William_Shatner],"[[{'docid': 'William_Shatner', 'end_sentence':...",William Shatner did not host a show that won a...,NaN
97954,99995,SUPPORTS,[Pablo_Fenjves],"[[{'docid': 'Pablo_Fenjves', 'end_sentence': 7...",Pablo Fenjves ghostwrote for a acting agent.,NaN
97955,99997,SUPPORTS,[Dwight_D._Eisenhower],"[[{'docid': 'Dwight_D._Eisenhower', 'end_sente...",The Republic of China was recognized as China'...,NaN


# DATA STATS

In [37]:
def generate_class_stats(train_df, test_df, val_df):
    text_lens_0 = []
    text_lens_1 = []
    rationale_lens_0 = []
    rationale_lens_1 = []
    rationale_percent_0 = []
    rationale_percent_1 = []
    class_distribution = [0,0]
    for df in [train_df, test_df, val_df]:
        for i in range(len(df)):
            df_row = df.loc[i]
            clas = df_row['classification']
            text = df_row['text']
            rationale = df_row['rationale']
            query = df_row['query']
            
            query_len = len(query.split())
            text_len = len(text.split()) - query_len - 1
            rationale_len = rationale.count(1) - query_len - 1
            rationale_percent = rationale_len/text_len
            if clas == "REFUTES":
                text_lens_0.append(text_len)
                rationale_lens_0.append(rationale_len)
                rationale_percent_0.append(rationale_percent)
                class_distribution[0] += 1
            else:
                text_lens_1.append(text_len)
                rationale_lens_1.append(rationale_len)
                rationale_percent_1.append(rationale_percent)
                class_distribution[1] += 1
                
    all_stats = {"text_lens_0": text_lens_0,
                 "text_lens_1": text_lens_1,
                 "text_lens_all":text_lens_0 + text_lens_1,
                 "rationale_lens_0":rationale_lens_0,
                 "rationale_lens_1":rationale_lens_1,
                 "rationale_lens_all":rationale_lens_0 + rationale_lens_1,
                 "rationale_percent_0": rationale_percent_0,
                 "rationale_percent_1": rationale_percent_1,
                 "rationale_percent_all": rationale_percent_0 + rationale_percent_1,
                 "class_distr":[class_distribution[0]/sum(class_distribution),class_distribution[1]/sum(class_distribution)]
                }
    return all_stats

In [38]:
all_stats = generate_class_stats(train_data_df,test_data_df,val_data_df)

In [39]:
for key,val in all_stats.items():
    print(f"{key}: {avg(val)}")
print(all_stats["class_distr"])

text_lens_0: 318.38970787736275
text_lens_1: 330.0212544956439
text_lens_all: 326.5197567837372
rationale_lens_0: 46.364957342256794
rationale_lens_1: 45.63103909424947
rationale_lens_all: 45.85197386332698
rationale_percent_0: 0.21115993968409894
rationale_percent_1: 0.19960476708895997
rationale_percent_all: 0.20308327357913616
class_distr: 0.5
[0.3010345766403485, 0.6989654233596515]


In [28]:
avg(rationale_percent_1)

0.22822182250835382